# This notebook contains tools and utilities for making and verifying custom slot machines for deployment on Ethereum.

In [5]:
import numpy as np
import itertools
from collections import Counter
import itertools

In [2]:
def makeSymbolList(numSymbolOnReel, numSymbolTotal, precision):
    fenceposts = [round(i, int(np.log10(precision))) for i in list(np.random.random(size=numSymbolOnReel - 1))]
    fenceposts = sorted([0] + fenceposts + [1])
    symbols = np.random.choice(numSymbolTotal, size=numSymbolOnReel, replace=False)
    symbolList = [(symbols[i],round((fenceposts[i+1] - fenceposts[i]), 
                                    int(np.log10(precision)))) for i in range(numSymbolOnReel)]
    return symbolList
    
def makeMachine(numSymbolList, numSymbolsTotal, precision):
    if numSymbolsTotal < max(numSymbolList):
        print("Total number of symbols too small...")
        return
    numReel = len(numSymbolList)
    machineDefList = [sorted(makeSymbolList(numSymbolList[i], 
                                            numSymbolsTotal, precision),
                             key=lambda x: x[1], reverse = True) for i in range(numReel)]
    print("Probabilities sum to 1? " + str([sum([j[1] for j in i]) for i in machineDefList]))
    return machineDefList

def evaluateMachine(machineDefList):
    numReel = len(machineDefList)
    probDict = {i:0 for i in range(1, numReel + 1)}
    for thisTuple in itertools.product(*machineDefList):
        #thisCount = numReel + 1 - len(set([i[0] for i in thisTuple]))
        thisCount = Counter([i[0] for i in thisTuple]).most_common()[0][1]
        thisProb = np.product([i[1] for i in thisTuple])
        if thisCount == numReel:
            probDict[str(thisTuple[0][0]) * numReel] = thisProb
        probDict[thisCount] += thisProb
    return probDict

def probDictToSolidity(probDict, payoutKeys):
    payoutString = ""
    numPayout = len(payoutKeys)
    maxMultiplier = 1;
    for thisKey in payoutKeys:
        thisProb = probDict[thisKey]
        thisMultiplier = int(1./(thisProb * numPayout))
        maxMultiplier = max(maxMultiplier, thisMultiplier)
        thisString = "    if (maxMatch == %s) {\n      return betAmount * %s;\n    }\n" % (thisKey, str(thisMultiplier))
        payoutString += thisString
    return (payoutString, maxMultiplier)

def makeMachineToSolidity(machine):
    flatMachine = [item for sublist in machine for item in sublist]
    precision = max([10**(len(str(i[1]).split(".")[1])) for i in flatMachine])
    solidityString = "function makeMachine() internal {\n"
    for index, reel in enumerate(machine):
        reelString0 = "    reels[%s].probDenominator = %s;\n" % (str(index), str(precision))
        reelString1 = "    reels[%s].probs = %s;\n" % (str(index), str([int(i[1] * precision) for i in reel]))
        reelString2 = "    reels[%s].eventLabels = %s;\n" % (str(index), str([i[0] for i in reel]))
        
        reelString = reelString0 + reelString1 + reelString2
        solidityString += reelString
    return solidityString + "}"

def makeSimpleMachine(numReels, numSymbols, probList):
    if sum(probList) - 1 > .00000001:
        print("Invalid probability list...")
        return None
    else:
        return [[(j,probList[j]) for j in range(numSymbols)] for i in range(numReels)]
    
def normalizeReel(reel):
    return [(reel.eventLabels[i], reel.probs[i]/float(reel.probDenominator)) for i in range(len(reel.eventLabels))]

def expectedValueForOutcome(normalizedOutcome, paytable):
    symbols, probs = zip(*normalizedOutcome)
    return paytable(symbols) * np.product(probs)
    
def expectedValue(reels, paytable):
    normalizedReels = [normalizeReel(reel) for reel in reels]
    allOutcomes = itertools.product(*normalizedReels)
    return sum([expectedValueForOutcome(thisOutcome, paytable) for thisOutcome in allOutcomes])

## Define slot machine

In [6]:
numSymbolOnReel = [9,9,9,9,9]
numSymbolTotal = 9
precision = 1000

numReel = len(numSymbolOnReel)

#machineOne = makeMachine(numSymbolOnReel, numSymbolTotal, precision)
machineOne = makeSimpleMachine(5, 9, [.12, .11, .11, .11, .11, .11, .11, .11, .11])

In [7]:
probDict = evaluateMachine(machineOne)
print(probDict)
print(sum([i[1] for i in probDict.items() if type(i[0]) == int]))

{1: 0.25580755200001704, 2: 0.6401897040001812, 3: 0.09772443999998831, 4: 0.0061245800000000105, 5: 0.000153724, '00000': 2.48832e-05, '11111': 1.61051e-05, '22222': 1.61051e-05, '33333': 1.61051e-05, '44444': 1.61051e-05, '55555': 1.61051e-05, '66666': 1.61051e-05, '77777': 1.61051e-05, '88888': 1.61051e-05}
1.0000000000001865


## Construct Solidity contract code to define the slot machine.

In [8]:
payoutString, maxMultiplier = probDictToSolidity(probDict,[4,5])

In [9]:
makeMachineString = makeMachineToSolidity(machineOne)

In [10]:
solidityCode = '''
// ******************************
// START MACHINE DEFINITION

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = %s;
  uint constant public maxMultipler = %s;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  %s
  
  // CALCULATE THE PAYOUT
  function paytable(uint[numReel] memory outcome, uint betAmount) internal returns (uint) {
    uint maxMatch = countMaxMatch(outcome);
%s}
  
// END MACHINE DEFINITION
// ***************************************
  
''' % (str(numReel), str(maxMultiplier), makeMachineString, payoutString)

## Print Solidity code for cut and paste into contract.

In [11]:
print(solidityCode)


// ******************************
// START MACHINE DEFINITION

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = 5;
  uint constant public maxMultipler = 3252;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  function makeMachine() internal {
    reels[0].probDenominator = 100;
    reels[0].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
    reels[0].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
    reels[1].probDenominator = 100;
    reels[1].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
    reels[1].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
    reels[2].probDenominator = 100;
    reels[2].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
    reels[2].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
    reels[3].probDenominator = 100;
    reels[3].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
    reels[3].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
    reels[4].probDenominator = 100;
    reels[4].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
    reels[4].eventLabels = 

# Code for verifying slot machine expected value

In [12]:
class Reel():
    def __init__(self):
        self.probDenominator = 0
        self.probs = []
        self.eventLabels = []
        
    def __repr__(self):
        return str((self.probDenominator, self.probs, self.eventLabels))

## Paste machine definition from contract here.

In [13]:
reels = [Reel() for i in range(5)]

reels[0].probDenominator = 100;
reels[0].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
reels[0].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
reels[1].probDenominator = 100;
reels[1].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
reels[1].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
reels[2].probDenominator = 100;
reels[2].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
reels[2].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
reels[3].probDenominator = 100;
reels[3].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
reels[3].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];
reels[4].probDenominator = 100;
reels[4].probs = [12, 11, 11, 11, 11, 11, 11, 11, 11];
reels[4].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7, 8];

reels

[(100, [12, 11, 11, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8]),
 (100, [12, 11, 11, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8]),
 (100, [12, 11, 11, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8]),
 (100, [12, 11, 11, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8]),
 (100, [12, 11, 11, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 6, 7, 8])]

## Define pyton paytable function by converting from Solidity paytable function in contract

In [17]:
'''
SOLIDITY PAYTABLE 

function paytable(uint[numReel] memory outcome, uint betAmount) internal returns (uint) {
    uint maxMatch = countMaxMatch(outcome);
    if (maxMatch == 4) {
      return betAmount * 81;
    }
    if (maxMatch == 5) {
      return betAmount * 3252;
    }
  }
'''

def paytable(outcome):
    
    def countMaxMatch(outcome):
        counter = Counter(outcome)
        return counter.most_common()[0][1]

    maxMatch = countMaxMatch(outcome)
    if (maxMatch == 4):
        return 81
    elif (maxMatch == 5):
        return 3252
    else:
        return 0


## Calculate expected value

In [16]:
expectedValue(reels, paytable)

0.9960014280000034